In [ ]:
from pathlib import Path
import pandas as pd
from matplotlib import pyplot as plt
import json

In [ ]:
import time
start_time = time.time()

In [ ]:
TRAIN_PATH = Path("/home/ubuntu/data-1/nybg2020/train/")

In [ ]:
! ls $TRAIN_PATH

In [ ]:
with open(TRAIN_PATH/"metadata.json", encoding="utf8", errors='ignore') as json_file:
    data = json.load(json_file)

In [ ]:
for i in data.keys():
    print(f'{i}: {len(data[i])}')

In [ ]:
df_ann =  pd.DataFrame.from_dict(data["annotations"])

In [ ]:
df_imgs =  pd.DataFrame.from_dict(data["images"])

In [ ]:
df_cat =  pd.DataFrame.from_dict(data["categories"])

In [ ]:
df_cat.head()

In [ ]:
df_merged = pd.merge(df_ann,df_imgs,on="id")
df_merged.to_csv("train_data.csv")
# df_merged = pd.merge(df_ann,df_imgs,left_on="image_id", right_on="id")

In [ ]:
df_merged = pd.merge(df_merged,df_cat, left_on="category_id", right_on="id")

In [ ]:
df_merged = df_merged.drop(columns=["id_x", "id_y"]) #get rid of repeated data

In [ ]:
# df_merged = df_merged.rename(columns={"category_id":"label", "file_name":"name", "name":"class_name"})

In [ ]:
 df_merged.head()

In [ ]:
family_ids_map = {n:i for i,n in enumerate(set(df_merged["family"]))}
family_ids = [family_ids_map[i] for i in list(df_merged["family"]) ]

In [ ]:
len(family_ids), len(df_merged)

In [ ]:
genus_ids_map = {n:i for i,n in enumerate(set(df_merged["genus"]))}
genus_ids = [genus_ids_map[i] for i in list(df_merged["genus"]) ]

In [ ]:
len(family_ids), len(df_merged)

In [ ]:
df_merged["family_id"] = family_ids
df_merged["genus_id"] = genus_ids

In [ ]:
df_merged.tail()

In [ ]:
# df_merged.to_csv(TRAIN_PATH/"train.csv")

In [ ]:
# df_merged["genus"].value_counts()

In [ ]:
# df_merged["family"].value_counts()

Look at one sample from each region
Might be an options to train region classifier that may help narrow down selecting which class it is
Triplet loss, maybe use look for whale tale for some ideas

In [ ]:
# sample = df_merged.sample(n=2000).drop_duplicates(['region_id'])
# f_names = list(sample["file_name"])

In [ ]:
# from PIL import Image

In [ ]:
# im0 = Image.open(TRAIN_PATH/f_names[0])
# im1 = Image.open(TRAIN_PATH/f_names[1])
# im2 = Image.open(TRAIN_PATH/f_names[2])
# im3 = Image.open(TRAIN_PATH/f_names[3])

In [ ]:
# f, axarr = plt.subplots(2,2)
# axarr[0,0].imshow(im0)
# axarr[0,1].imshow(im1)
# axarr[1,0].imshow(im2)
# axarr[1,1].imshow(im3)


In [ ]:
#plt.imshow(im1)

Game plan
- pretrain on identifying region ?
- look at techniquqes used in the whawle identification stuff
- possibly use simese network

In [13]:
from fastai.vision import *

In [14]:
TRAIN_PATH = Path("/home/shaun/personal/kaggle-data/nybg2020/train/")
df = pd.read_csv(TRAIN_PATH/"train.csv")
df_train = df[[ "file_name", "category_id"]]

In [15]:
# df_train = df_merged[["file_name", "family_id"]]

In [16]:
df_train.head()

,file_name,category_id
0,images/156/72/354106.jpg,15672
1,images/156/72/545181.jpg,15672
2,images/156/72/449419.jpg,15672
3,images/156/72/200223.jpg,15672
4,images/156/72/5327.jpg,15672


In [17]:
df_train = df_train.rename(columns={"category_id": "label", "file_name":" name"})

In [18]:
def sample_class(x):
    if len(x) < 10:
        return x.sample(n=10, replace=True) 
    return x

In [19]:
# counts  = df_train.label.value_counts() # Get Class counts

# labels_keep = counts.keys()[counts>9] # Get labels that have more then 9 exmaples

# df_new_train = df_train[df_train.label.isin(labels_keep)] # Select those items from df

# # Do a random sample from data frame of 20 items
# new_df = df_new_train.groupby('label').apply(lambda x: x.sample(n=20, replace=True)).reset_index(drop = True)

In [20]:
# TRAIN_PATH = Path("/home/shaun/personal/kaggle-data/nybg2020/train/")
# df = pd.read_csv(TRAIN_PATH/"train.csv")
# df_train = df[[ "file_name", "category_id"]]

In [21]:
new_df = df_train.groupby('label').apply(lambda x: x.sample(n=1)).reset_index(drop = True)

In [22]:
# new_df = df_train

In [23]:
len(new_df)

32093

In [24]:
# there is a better way to do this
def sample(df, n=4):   
    validation_set = {}
    bool_arr = []
    for i,row in df.iterrows():
        if validation_set.get(row["label"], 0) == n:
            bool_arr.append(False) 
        else:
            validation_set[row["label"]] = validation_set.get(row["label"], 0) + 1
            bool_arr.append(True) 

        
    return bool_arr

In [25]:
# valid = sample(new_df, n=4)

In [26]:
# new_df["is_valid"] = valid

In [27]:
new_df.tail()

,name,label
32088,images/320/89/96454.jpg,32089
32089,images/320/90/808278.jpg,32090
32090,images/320/91/838639.jpg,32091
32091,images/320/92/38384.jpg,32092
32092,images/320/93/1010199.jpg,32093


In [28]:
classes = new_df["label"].unique()

In [29]:
classes

array([    0,     1,     2,     3, ..., 32090, 32091, 32092, 32093])

In [30]:
len(classes)

32093

In [31]:
classes_map = {classes[i]:i for i in range(len(classes))}

In [32]:
classes_map_key = {v: k for k,v in classes_map.items()}

In [33]:
classes_map[15668]

15668

In [34]:
classes_map[32092]

32091

In [35]:
classes_map_key[19337]

19337

In [24]:
new_labels = [classes_map[i] for i in new_df["label"]]

In [25]:
new_df["new_labels"] = new_labels

In [26]:
data = (ImageList.from_df(new_df,TRAIN_PATH)
        .split_none()
        .label_from_df(cols="new_labels")
        .transform(size=256)
        .databunch(bs=64))

In [27]:
# data.show_batch()

In [28]:
learn = cnn_learner(data, models.resnet34, metrics=accuracy).to_fp16()

In [29]:
# learn

In [30]:
# learn.save("test",return_path=True)

In [31]:
# learn.fit_one_cycle(3)

In [32]:
# learn.save("model-all-classes")

In [33]:
# learn.unfreeze()

In [34]:
# learn.fit_one_cycle(3, max_lr=(1e-4, 1e-3, 1e-2), wd=(1e-4,1e-4,1e-1))

In [35]:
# learn.save("model-all-classes-unfreeze")

In [36]:
# learn.fit_one_cycle(3, max_lr=(1e-4, 1e-3, 1e-2), wd=(1e-4,1e-4,1e-1))

In [37]:
# learn.save("model-half-classes-unfreeze-2")

## Submission

In [38]:
learn.load("/home/shaun/personal/kaggle-data/nybg2020/models/model-all-classes-unfreeze")
1

1

In [4]:
TEST_PATH = Path("/home/shaun/personal/kaggle-data/nybg2020/test/")


In [37]:
# get test data
with open(TEST_PATH/"metadata.json", encoding="utf8", errors='ignore') as json_file:
    data = json.load(json_file)
df_test =  pd.DataFrame.from_dict(data["images"])

In [40]:
ds1, ds2 = df_test[:len(df_test)//2], df_test[len(df_test)//2:]

In [43]:
ds1_1, ds2_1 = ds1[:len(ds1)//2], ds1[len(ds1)//2:]

In [41]:
ds1_2, ds2_2 = ds2[:len(ds2)//2], ds2[len(ds2)//2:]

In [42]:
test_data = ImageImageList.from_df(ds2_2, TEST_PATH, cols="file_name")
learn.data.add_test(test_data)

In [43]:
preds, y = learn.get_preds(ds_type=DatasetType.Test)

In [44]:
len(preds), len(ds1)

(34573, 69146)

In [7]:
# ds1_1 = np.argmax(preds, axis=1)
a = np.load( TEST_PATH/'ds1_1.npy')
a.shape

(34573,)

In [8]:
# ds2_1 = np.argmax(preds, axis=1)
b = np.load(TEST_PATH/'ds2_1.npy')

In [9]:
# ds1_2 = np.argmax(preds, axis=1)
c = np.load( TEST_PATH/'ds1_2.npy')

In [10]:
# ds2_2 = np.argmax(preds, axis=1)
d = np.load(TEST_PATH/'ds2_2.npy')

In [11]:
joined = np.concatenate((a, b,c,d), axis=None)

In [36]:
res_correct =  [classes_map_key[i] for i in joined]

In [38]:
df_test["Predicted"] = res_correct

In [ ]:
#learn.summary

In [39]:
sub = df_test[["id", "Predicted"]]

In [40]:
subm_file = "/home/shaun/personal/kaggle-data/nybg2020/sub-3"

In [41]:
sub.to_csv(subm_file, index=False)

In [42]:
! kaggle competitions submit herbarium-2020-fgvc7 -f $subm_file -m "fastai sub"

100%|███████████████████████████████████████| 1.56M/1.56M [00:08<00:00, 191kB/s]
Successfully submitted to Herbarium 2020 - FGVC7